In [37]:
import pandas as pd

# Load the Excel file
df = pd.read_excel("raw_data/20251024_raw data BioStruct_map client project_final.xlsx")

# Clean all string columns: normalize and map unsuitable values to "unknown"
def clean_string_columns(dataframe):
    """Clean string columns, normalizing whitespace/newlines and mapping unusable values to 'unknown'."""
    import re
    for column in dataframe.columns:
        if dataframe[column].dtype == 'object':  # String columns
            # Start with null mask from original series
            null_mask = dataframe[column].isna()
            ser = dataframe[column].astype(str)
            # Normalize whitespace and newlines
            ser = ser.str.replace('\n', ' ', regex=False).str.strip()
            ser_lower = ser.str.casefold().str.strip()
            # Build masks for unsuitable tokens
            blank_or_dash = ser.str.fullmatch(r"\s*") | ser.str.fullmatch(r"[-–—]+")
            na_like = ser_lower.str.fullmatch(r"n[\W_]*a(?:vailable)?")
            none_like = ser_lower.str.fullmatch(r"none")
            null_like = ser_lower.str.fullmatch(r"null")
            unknown_like = ser_lower.str.fullmatch(r"unknown")
            nan_like = ser_lower.str.fullmatch(r"nan")
            unsuitable_mask = null_mask | blank_or_dash | na_like | none_like | null_like | unknown_like | nan_like
            ser = ser.mask(unsuitable_mask, 'unknown')
            dataframe[column] = ser
    return dataframe

# Apply cleaning to all columns
df = clean_string_columns(df)

print("Data loaded and cleaned:")
print(f"Shape: {df.shape}")
print(f"Columns: {list(df.columns)}")
df.head()


Data loaded and cleaned:
Shape: (308, 9)
Columns: ['ID', 'Workshop', 'Evaluation', 'Gender', 'Status', 'Scientific_Discipline', 'Institutional_ID', 'Primary_ institutional_affiliation', 'Country_of_primary_affiliation']


ID  Workshop Evaluation Gender       Status  \
0  9208  Cameroon          R   Male      Pre-Doc   
1  4358     Kenya          R   Male      Pre-Doc   
2  7084     Kenya          S   Male  PhD Student   
3  8849      Mali          R   Male      Pre-Doc   
4  3158     Kenya          R   Male      Pre-Doc   

                               Scientific_Discipline  Institutional_ID  \
0                                  Organic Chemistry               NaN   
1                                           Pharmacy               NaN   
2                                Medicinal Chemistry               NaN   
3                                            unknown               NaN   
4  Bioinformatics, Zoology, Cell Biology and Gene...               NaN   

                  Primary_ institutional_affiliation  \
0          Prince Abubakar  Audu University, Nigeria   
1                                            unknown   
2                Nazi Boni University,  Burkina Faso   
3  Université des Sciences, des Techniques et des...   
4                    Lagos State University, Nigeria   

  Country_of_primary_affiliation  
0                        Nigeria  
1                        unknown  
2                   Burkina Faso  
3                           Mali  
4                        Nigeria

In [38]:
df

ID  Workshop Evaluation  Gender       Status  \
0    9208  Cameroon          R    Male      Pre-Doc   
1    4358     Kenya          R    Male      Pre-Doc   
2    7084     Kenya          S    Male  PhD Student   
3    8849      Mali          R    Male      Pre-Doc   
4    3158     Kenya          R    Male      Pre-Doc   
..    ...       ...        ...     ...          ...   
303  1011      Mali          S    Male      Pre-Doc   
304  5626     Kenya          R  Female      Pre-Doc   
305  1727     Kenya          R    Male      Pre-Doc   
306  8008     Kenya          S  Female  PhD Student   
307  5825     Kenya          S    Male     Post-Doc   

                                 Scientific_Discipline  Institutional_ID  \
0                                    Organic Chemistry               NaN   
1                                             Pharmacy               NaN   
2                                  Medicinal Chemistry               NaN   
3                                              unknown               NaN   
4    Bioinformatics, Zoology, Cell Biology and Gene...               NaN   
..                                                 ...               ...   
303                                 Structural Biology               NaN   
304                    Bioinformatics and Biochemistry               NaN   
305  Medical Laboratories  Sciences and  Microbiolo...               NaN   
306                                      Biotechnology               NaN   
307       Medical Microbiology and Infectious Diseases               NaN   

                    Primary_ institutional_affiliation  \
0            Prince Abubakar  Audu University, Nigeria   
1                                              unknown   
2                  Nazi Boni University,  Burkina Faso   
3    Université des Sciences, des Techniques et des...   
4                      Lagos State University, Nigeria   
..                                                 ...   
303                                            unknown   
304                            -, United Arab Emriates   
305                 Alzaiem Alazhari University, Sudan   
306                   Ahmadu Bello University, Nigeria   
307                      University of Nairobi,  Kenya   

    Country_of_primary_affiliation  
0                          Nigeria  
1                          unknown  
2                     Burkina Faso  
3                             Mali  
4                          Nigeria  
..                             ...  
303                        unknown  
304           United Arab Emirates  
305                          Sudan  
306                        Nigeria  
307                          Kenya  

[308 rows x 9 columns]

In [39]:
# Apply additional cleaning to the main dataset
def clean_string_columns(dataframe):
    """Clean string columns, normalizing whitespace/newlines and mapping unusable values to 'unknown'."""
    print("Cleaning string columns...")
    unsuitable_values = {"", "-", "na", "n/a", "null", "none"}
    for column in dataframe.columns:
        if dataframe[column].dtype == 'object':  # String columns
            ser = dataframe[column].astype(str)
            ser = ser.str.replace('\n', ' ', regex=False).str.strip()
            ser_lower = ser.str.lower().str.strip()
            ser = ser.where(~ser_lower.isin(unsuitable_values) & (ser_lower.str.len() > 0), 'unknown')
            dataframe[column] = ser
    return dataframe

# Apply cleaning to the main dataset
df = clean_string_columns(df)

print(f"\nDataset after cleaning:")
print(f"Shape: {df.shape}")
print("\nFirst few rows after cleaning:")
df.head()


Cleaning string columns...

Dataset after cleaning:
Shape: (308, 9)

First few rows after cleaning:


ID  Workshop Evaluation Gender       Status  \
0  9208  Cameroon          R   Male      Pre-Doc   
1  4358     Kenya          R   Male      Pre-Doc   
2  7084     Kenya          S   Male  PhD Student   
3  8849      Mali          R   Male      Pre-Doc   
4  3158     Kenya          R   Male      Pre-Doc   

                               Scientific_Discipline  Institutional_ID  \
0                                  Organic Chemistry               NaN   
1                                           Pharmacy               NaN   
2                                Medicinal Chemistry               NaN   
3                                            unknown               NaN   
4  Bioinformatics, Zoology, Cell Biology and Gene...               NaN   

                  Primary_ institutional_affiliation  \
0          Prince Abubakar  Audu University, Nigeria   
1                                            unknown   
2                Nazi Boni University,  Burkina Faso   
3  Université des Sciences, des Techniques et des...   
4                    Lagos State University, Nigeria   

  Country_of_primary_affiliation  
0                        Nigeria  
1                        unknown  
2                   Burkina Faso  
3                           Mali  
4                        Nigeria

In [40]:
# Clean 'Primary_ institutional_affiliation' column: remove country names
def remove_country_from_affiliation(affiliation):
    if isinstance(affiliation, str) and ',' in affiliation:
        return affiliation.split(',')[0].strip()
    return affiliation

df['Primary_ institutional_affiliation'] = df['Primary_ institutional_affiliation'].apply(remove_country_from_affiliation)
df.head()


ID  Workshop Evaluation Gender       Status  \
0  9208  Cameroon          R   Male      Pre-Doc   
1  4358     Kenya          R   Male      Pre-Doc   
2  7084     Kenya          S   Male  PhD Student   
3  8849      Mali          R   Male      Pre-Doc   
4  3158     Kenya          R   Male      Pre-Doc   

                               Scientific_Discipline  Institutional_ID  \
0                                  Organic Chemistry               NaN   
1                                           Pharmacy               NaN   
2                                Medicinal Chemistry               NaN   
3                                            unknown               NaN   
4  Bioinformatics, Zoology, Cell Biology and Gene...               NaN   

  Primary_ institutional_affiliation Country_of_primary_affiliation  
0   Prince Abubakar  Audu University                        Nigeria  
1                            unknown                        unknown  
2               Nazi Boni University                   Burkina Faso  
3            Université des Sciences                           Mali  
4             Lagos State University                        Nigeria

In [41]:
# Display columns to check for country information
print(df.columns)


Index(['ID', 'Workshop', 'Evaluation', 'Gender', 'Status',
       'Scientific_Discipline', 'Institutional_ID',
       'Primary_ institutional_affiliation', 'Country_of_primary_affiliation'],
      dtype='object')


In [42]:
df.drop('Institutional_ID', axis=1, inplace=True)

In [43]:
# Check unique values in 'Primary_ institutional_affiliation' to understand standardization needs
print(df['Primary_ institutional_affiliation'].nunique())
print(df['Primary_ institutional_affiliation'].unique()[:50]) # Displaying first 50 unique values

# Assuming there is a 'Country' column based on the request to map countries to regions
# Let's check for a 'Country' column or similar to extract country information
# If no explicit 'Country' column, I'll have to infer it from 'Primary_ institutional_affiliation' or ask for clarification.
# For now, let's assume a 'Country' column exists or can be derived.

# Placeholder for country extraction (if needed and no explicit country column exists)
# df['Country'] = df['Primary_ institutional_affiliation'].apply(lambda x: x.split(',')[-1].strip() if isinstance(x, str) and ',' in x else '')
# print(df['Country'].unique())


166
['Prince Abubakar  Audu University' 'unknown' 'Nazi Boni University'
 'Université des Sciences' 'Lagos State University'
 'Black Walls Systems and Consultancy' 'Ahmadu Bello University'
 'KolaDaisi University' 'Bowen University' 'University of Yaoundé I'
 'University Alzaiem Alazhari'
 'Kenya Industrial Research  Development Institute'
 'University of Abomey-calavi' 'Alaanan Veterinary Clinic for Animals'
 'Darfur Next Generation  Organization' 'Hokkaido University'
 'Egypt-japanese University  for Science and Technology'
 'Ain Shams University' 'Target Malaria' 'University NANGUI  ABROGOUA'
 'University Nangui  Abrogoua' 'Khartoum Oncology Hospital'
 'University of Bakht Alruda' 'University of Science and  Technology'
 'Cairo University' 'Omdurman Islamic University' 'Kenya Water Institute'
 'Addis Ababa University' 'University of Sadat City'
 'Institut Pasteur  De Dakar'
 'Kwame Nkrumah University of Science and Technology'
 'University of Abomey- Calavi' 'University of Dschang'


In [44]:
# Define a mapping of countries to regions
region_mapping = {
    'Benin': 'Africa',
    'Botswana': 'Africa',
    'Cameroon': 'Africa',
    'Egypt': 'Africa',
    'Ethiopia': 'Africa',
    'Ghana': 'Africa',
    'Ivory Coast': 'Africa',
    'Kenya': 'Africa',
    'Nigeria': 'Africa',
    'Spain': 'Europe',
    'Sudan': 'Africa',
    'Tanzania': 'Africa',
    'Uganda': 'Africa',
    'Zambia': 'Africa',
    'Burkina Faso': 'Africa',
    'Burundi': 'Africa',
    'China': 'Asia',
    'Gambia': 'Africa',
    'Germany': 'Europe',
    'India': 'Asia',
    'Italy': 'Europe',
    'Japan': 'Asia',
    'Mali': 'Africa',
    'Qatar': 'Middle-east',
    'Russia': 'Europe',
    'Saudi Arabia': 'Middle-east',
    'Senegal': 'Africa',
    'South Africa': 'Africa',
    'Tchad': 'Africa',
    'The Netherlands': 'Europe',
    'Tunisia': 'Africa',
    'United Arab Emirates': 'Middle-east',
    'United Kingdom': 'Europe',
    'Zimbabwe': 'Africa',
}

# Apply the mapping to create the 'region' column using the 'Country' column
df['region'] = df['Country_of_primary_affiliation'].map(region_mapping).fillna('unknown')
df.head()


ID  Workshop Evaluation Gender       Status  \
0  9208  Cameroon          R   Male      Pre-Doc   
1  4358     Kenya          R   Male      Pre-Doc   
2  7084     Kenya          S   Male  PhD Student   
3  8849      Mali          R   Male      Pre-Doc   
4  3158     Kenya          R   Male      Pre-Doc   

                               Scientific_Discipline  \
0                                  Organic Chemistry   
1                                           Pharmacy   
2                                Medicinal Chemistry   
3                                            unknown   
4  Bioinformatics, Zoology, Cell Biology and Gene...   

  Primary_ institutional_affiliation Country_of_primary_affiliation   region  
0   Prince Abubakar  Audu University                        Nigeria   Africa  
1                            unknown                        unknown  unknown  
2               Nazi Boni University                   Burkina Faso   Africa  
3            Université des Sciences                           Mali   Africa  
4             Lagos State University                        Nigeria   Africa

In [45]:
# Standardize institution names and correct typos
standardization_map = {
    'Prince Abubakar\n Audu University': 'Prince Abubakar Audu University',
    'Nazi Boni University': 'Nazi Boni University',
    'Université des Sciences': 'Université des Sciences',
    'Lagos State University': 'Lagos State University',
    'Black Walls Systems\nand Consultancy': 'Black Walls Systems and Consultancy',
    'Ahmadu Bello University': 'Ahmadu Bello University',
    'KolaDaisi University': 'KolaDaisi University',
    'Bowen University': 'Bowen University',
    'University of Yaoundé I': 'University of Yaoundé I',
    'University Alzaiem Alazhari': 'University Alzaiem Alazhari',
    'Kenya Industrial Research\n Development Institute': 'Kenya Industrial Research Development Institute',
    'University of Abomey-calavi': 'University of Abomey-Calavi',
    'Alaanan Veterinary Clinic\nfor Animals': 'Alaanan Veterinary Clinic for Animals',
    'Darfur Next Generation\n Organization': 'Darfur Next Generation Organization',
    'Hokkaido\nUniversity': 'Hokkaido University',
    'Egypt-japanese University\n for Science and Technology': 'Egypt-Japanese University for Science and Technology',
    'Ain Shams University': 'Ain Shams University',
    'Target Malaria': 'Target Malaria',
    'University NANGUI \nABROGOUA': 'University Nangui Abrogoua',
    'University Nangui \nAbrogoua': 'University Nangui Abrogoua',
    'Khartoum Oncology\nHospital': 'Khartoum Oncology Hospital',
    'University of Bakht\nAlruda': 'University of Bakht Alruda',
    'University of Science and\n Technology': 'University of Science and Technology',
    'Cairo University': 'Cairo University',
    'Omdurman Islamic\nUniversity': 'Omdurman Islamic University',
    'Kenya Water Institute': 'Kenya Water Institute',
    'Addis Ababa University': 'Addis Ababa University',
    'University of Sadat City': 'University of Sadat City',
    'Institut Pasteur\n De Dakar': 'Institut Pasteur De Dakar',
    'Kwame Nkrumah University\nof Science and Technology': 'Kwame Nkrumah University of Science and Technology',
    'University of Abomey-\nCalavi': 'University of Abomey-Calavi',
    'University of Dschang': 'University of Dschang',
    'University of Ilorin': 'University of Ilorin',
    'University of Zimbabwe': 'University of Zimbabwe',
    'Children Cancer\n Hospital 57357': 'Children Cancer Hospital 57357',
    'Karary University': 'Karary University',
    'Badr University': 'Badr University',
    'Qatar': 'Qatar University',
    'Redeemer’s University': 'Redeemer’s University',
    'University of Health and Allied Sciences\xa0(UHAS)': 'University of Health and Allied Sciences (UHAS)',
    'Murang’a University of\n Sciences and\n Technology': 'Murang’a University of Sciences and Technology',
    'University of Bamenda': 'University of Bamenda',
    'Pwani University': 'Pwani University',
    'KU Leuven': 'KU Leuven',
    'Pan African University Life\n and  Earth Sciences\n Institute (including\n Health and Agriculture)': 'Pan African University Life and Earth Sciences Institute (including Health and Agriculture)',
    'Covenant University': 'Covenant University',
    'University of Douala': 'University of Douala',
    'Kenyata University': 'Kenyatta University',
    'Botswana International\n University of Science And\n Technology': 'Botswana International University of Science and Technology',
    # Add more standardization rules as needed
}

def standardize_affiliation(affiliation):
    if isinstance(affiliation, str):
        for old, new in standardization_map.items():
            affiliation = affiliation.replace(old, new)
    return affiliation

df['Primary_ institutional_affiliation'] = df['Primary_ institutional_affiliation'].apply(standardize_affiliation)
df.head()


ID  Workshop Evaluation Gender       Status  \
0  9208  Cameroon          R   Male      Pre-Doc   
1  4358     Kenya          R   Male      Pre-Doc   
2  7084     Kenya          S   Male  PhD Student   
3  8849      Mali          R   Male      Pre-Doc   
4  3158     Kenya          R   Male      Pre-Doc   

                               Scientific_Discipline  \
0                                  Organic Chemistry   
1                                           Pharmacy   
2                                Medicinal Chemistry   
3                                            unknown   
4  Bioinformatics, Zoology, Cell Biology and Gene...   

  Primary_ institutional_affiliation Country_of_primary_affiliation   region  
0   Prince Abubakar  Audu University                        Nigeria   Africa  
1                            unknown                        unknown  unknown  
2               Nazi Boni University                   Burkina Faso   Africa  
3            Université des Sciences                           Mali   Africa  
4             Lagos State University                        Nigeria   Africa

In [46]:
# Perform statistical summary for exploratory data analysis
print(df.describe(include='all'))
print("\nValue counts for 'region':\n", df['region'].value_counts())
print("\nValue counts for 'Primary_ institutional_affiliation':\n", df['Primary_ institutional_affiliation'].value_counts().head(20)) # Display top 20 affiliations


                 ID Workshop Evaluation Gender   Status Scientific_Discipline  \
count    308.000000      308        308    308      308                   308   
unique          NaN        3          2      2        8                   166   
top             NaN    Kenya          R   Male  Pre-Doc          Biochemistry   
freq            NaN      173        198    201      144                    56   
mean    5292.487013      NaN        NaN    NaN      NaN                   NaN   
std     2677.037385      NaN        NaN    NaN      NaN                   NaN   
min     1008.000000      NaN        NaN    NaN      NaN                   NaN   
25%     2801.000000      NaN        NaN    NaN      NaN                   NaN   
50%     5192.000000      NaN        NaN    NaN      NaN                   NaN   
75%     7757.500000      NaN        NaN    NaN      NaN                   NaN   
max     9988.000000      NaN        NaN    NaN      NaN                   NaN   

       Primary_ institution

In [47]:
# Load and examine the new mapping file
import pandas as pd
import re

# Read the OECD mapping file
mapping_df = pd.read_excel("raw_data/OECD schema to Web of Science Category Mapping 2022.xlsx")
print("Original mapping file shape:", mapping_df.shape)
print("\nFirst few rows:")
print(mapping_df.head(10))
print("\nColumn names:")
print(mapping_df.columns.tolist())

# Clean the mapping file
def clean_oecd_category(category):
    """Remove number prefix and convert to lowercase"""
    if isinstance(category, str):
        # Remove number prefix (e.g., "1 NATURAL SCIENCES" -> "natural sciences")
        # cleaned = re.sub(r'^\d+\s+', '', category)
        cleaned = ' '.join(category.split(' ')[1:])
        return cleaned.lower()
    return category

# Apply cleaning to the oecddesc column
mapping_df['major_category'] = mapping_df['oecddesc'].apply(clean_oecd_category)

# Also clean the wosdesc column to lowercase
mapping_df['wos_category'] = mapping_df['wosdesc'].str.lower()

# Create a clean mapping dataframe with only the columns we need
clean_mapping_df = mapping_df[['major_category', 'wos_category']].copy()

print("\nCleaned mapping file:")
print(clean_mapping_df.head(10))
print(f"\nUnique major categories: {clean_mapping_df['major_category'].nunique()}")
print("Major categories:")
print(clean_mapping_df['major_category'].unique())


Original mapping file shape: (521, 2)

First few rows:
             oecddesc                                           wosdesc
0  1 NATURAL SCIENCES                                             LOGIC
1  1 NATURAL SCIENCES                              MATHEMATICS, APPLIED
2  1 NATURAL SCIENCES       MATHEMATICS, INTERDISCIPLINARY APPLICATIONS
3  1 NATURAL SCIENCES                                       MATHEMATICS
4  1 NATURAL SCIENCES                             PHYSICS, MATHEMATICAL
5  1 NATURAL SCIENCES                          STATISTICS & PROBABILITY
6  1 NATURAL SCIENCES         COMPUTER SCIENCE, ARTIFICIAL INTELLIGENCE
7  1 NATURAL SCIENCES                     COMPUTER SCIENCE, CYBERNETICS
8  1 NATURAL SCIENCES             COMPUTER SCIENCE, INFORMATION SYSTEMS
9  1 NATURAL SCIENCES  COMPUTER SCIENCE, INTERDISCIPLINARY APPLICATIONS

Column names:
['oecddesc', 'wosdesc']

Cleaned mapping file:
     major_category                                      wos_category
0  natural sciences  

In [48]:
# Apply cleaning to the mapping file as well
print("Cleaning mapping file...")
mapping_df = clean_string_columns(mapping_df)

print("Mapping file cleaned successfully!")
print("\nFirst few rows of cleaned mapping file:")
print(mapping_df.head())

Cleaning mapping file...
Cleaning string columns...
Mapping file cleaned successfully!

First few rows of cleaned mapping file:
             oecddesc                                      wosdesc  \
0  1 NATURAL SCIENCES                                        LOGIC   
1  1 NATURAL SCIENCES                         MATHEMATICS, APPLIED   
2  1 NATURAL SCIENCES  MATHEMATICS, INTERDISCIPLINARY APPLICATIONS   
3  1 NATURAL SCIENCES                                  MATHEMATICS   
4  1 NATURAL SCIENCES                        PHYSICS, MATHEMATICAL   

     major_category                                 wos_category  
0  natural sciences                                        logic  
1  natural sciences                         mathematics, applied  
2  natural sciences  mathematics, interdisciplinary applications  
3  natural sciences                                  mathematics  
4  natural sciences                        physics, mathematical  


In [49]:
# Export cleaned mapping to processed_data folder
import os

# Create processed_data directory if it doesn't exist
os.makedirs("processed_data", exist_ok=True)

# Export the cleaned mapping
clean_mapping_df.to_csv("processed_data/cleaned_oecd_mapping.csv", index=False)

print("Cleaned mapping exported to processed_data folder:")
print("- cleaned_oecd_mapping.csv")
print(f"\nTotal mappings: {len(clean_mapping_df)}")
print(f"Unique major categories: {clean_mapping_df['major_category'].nunique()}")


Cleaned mapping exported to processed_data folder:
- cleaned_oecd_mapping.csv

Total mappings: 521
Unique major categories: 48


In [50]:
# Add fuzzy matching for scientific disciplines
from fuzzywuzzy import fuzz, process
import numpy as np

def find_best_major_category(scientific_discipline, mapping_df, partial_threshold=60):
    """
    Find the best matching major category using the following order:
    1. If any wos_category is a substring of the discipline, use its major_category.
    2. Else if the discipline is a substring of any wos_category, use its major_category.
    3. Else use fuzzy matching as a fallback (threshold-based).
    4. Return 'unknown' if no good match.
    """
    # Normalize input
    if scientific_discipline is None:
        return 'unknown'
    discipline_lower = str(scientific_discipline).lower().strip()
    if discipline_lower in {"", "-", "na", "n/a", "null", "none", "unknown"}:
        return 'unknown'

    # Candidate list
    wos_categories = mapping_df['wos_category'].tolist()

    # 1) wos_category is substring of discipline
    subs_candidates_1 = [w for w in wos_categories if w in discipline_lower]
    if subs_candidates_1:
        # Prefer the longest substring match
        best_wos = max(subs_candidates_1, key=len)
        return mapping_df[mapping_df['wos_category'] == best_wos]['major_category'].iloc[0]

    # 2) discipline is substring of wos_category
    subs_candidates_2 = [w for w in wos_categories if discipline_lower in w]
    if subs_candidates_2:
        # Prefer the shortest containing string (closest match)
        best_wos = min(subs_candidates_2, key=len)
        return mapping_df[mapping_df['wos_category'] == best_wos]['major_category'].iloc[0]

    # 3) Fuzzy fallback
    best_match = process.extractOne(discipline_lower, wos_categories, scorer=fuzz.token_sort_ratio)
    if best_match and best_match[1] >= partial_threshold:
        matched_wos = best_match[0]
        return mapping_df[mapping_df['wos_category'] == matched_wos]['major_category'].iloc[0]

    # 4) Unknown
    return 'unknown'

# Apply matching to create the major_scientific_discipline column
print("Applying matching to scientific disciplines...")
print("1. Substring (wos in discipline)")
print("2. Substring (discipline in wos)")
print("3. Fuzzy fallback")
print("4. Unknown if nothing matches")
print()

df['major_scientific_discipline'] = df['Scientific_Discipline'].apply(
    lambda x: find_best_major_category(x, clean_mapping_df)
)

# Show results
print("\nFuzzy matching results:")
print(df['major_scientific_discipline'].value_counts())
print(f"\nTotal records: {len(df)}")
unknown_count = sum(df['major_scientific_discipline'] == 'unknown')
matched_count = sum(df['major_scientific_discipline'] != 'unknown')
print(f"Unknown matches: {unknown_count}")
print(f"Successfully matched: {matched_count}")
print(f"Match rate: {matched_count/len(df)*100:.1f}%")


Applying matching to scientific disciplines...
1. Substring (wos in discipline)
2. Substring (discipline in wos)
3. Fuzzy fallback
4. Unknown if nothing matches


Fuzzy matching results:
major_scientific_discipline
natural sciences                        178
medical and health sciences              67
unknown                                  27
engineering and technology               27
social sciences                           5
agricultural and veterinary sciences      4
Name: count, dtype: int64

Total records: 308
Unknown matches: 27
Successfully matched: 281
Match rate: 91.2%


In [51]:
df

ID  Workshop Evaluation  Gender       Status  \
0    9208  Cameroon          R    Male      Pre-Doc   
1    4358     Kenya          R    Male      Pre-Doc   
2    7084     Kenya          S    Male  PhD Student   
3    8849      Mali          R    Male      Pre-Doc   
4    3158     Kenya          R    Male      Pre-Doc   
..    ...       ...        ...     ...          ...   
303  1011      Mali          S    Male      Pre-Doc   
304  5626     Kenya          R  Female      Pre-Doc   
305  1727     Kenya          R    Male      Pre-Doc   
306  8008     Kenya          S  Female  PhD Student   
307  5825     Kenya          S    Male     Post-Doc   

                                 Scientific_Discipline  \
0                                    Organic Chemistry   
1                                             Pharmacy   
2                                  Medicinal Chemistry   
3                                              unknown   
4    Bioinformatics, Zoology, Cell Biology and Gene...   
..                                                 ...   
303                                 Structural Biology   
304                    Bioinformatics and Biochemistry   
305  Medical Laboratories  Sciences and  Microbiolo...   
306                                      Biotechnology   
307       Medical Microbiology and Infectious Diseases   

    Primary_ institutional_affiliation Country_of_primary_affiliation  \
0     Prince Abubakar  Audu University                        Nigeria   
1                              unknown                        unknown   
2                 Nazi Boni University                   Burkina Faso   
3              Université des Sciences                           Mali   
4               Lagos State University                        Nigeria   
..                                 ...                            ...   
303                            unknown                        unknown   
304                                  -           United Arab Emirates   
305        Alzaiem Alazhari University                          Sudan   
306            Ahmadu Bello University                        Nigeria   
307              University of Nairobi                          Kenya   

          region  major_scientific_discipline  
0         Africa             natural sciences  
1        unknown  medical and health sciences  
2         Africa  medical and health sciences  
3         Africa                      unknown  
4         Africa             natural sciences  
..           ...                          ...  
303      unknown             natural sciences  
304  Middle-east             natural sciences  
305       Africa             natural sciences  
306       Africa   engineering and technology  
307       Africa  medical and health sciences  

[308 rows x 10 columns]

In [35]:
df['ID'].nunique()

286

In [52]:
# Export the final cleansed data with the new major_scientific_discipline column

df.to_csv("processed_data/cleansed_biostruct_data_v2.csv", index=False)

print("Final cleansed data exported to processed_data folder:")

print("- cleansed_biostruct_data_with_major_categories.csv")

print(f"\nDataset summary:")
print(f"Total records: {len(df)}")
print(f"Columns: {list(df.columns)}")
print(f"\nMajor scientific discipline distribution:")
print(df['major_scientific_discipline'].value_counts())

print(f"\nRegion distribution:")
print(df['region'].value_counts())



Final cleansed data exported to processed_data folder:
- cleansed_biostruct_data_with_major_categories.csv

Dataset summary:
Total records: 308
Columns: ['ID', 'Workshop', 'Evaluation', 'Gender', 'Status', 'Scientific_Discipline', 'Primary_ institutional_affiliation', 'Country_of_primary_affiliation', 'region', 'major_scientific_discipline']

Major scientific discipline distribution:
major_scientific_discipline
natural sciences                        178
medical and health sciences              67
unknown                                  27
engineering and technology               27
social sciences                           5
agricultural and veterinary sciences      4
Name: count, dtype: int64

Region distribution:
region
Africa         277
unknown         14
Asia             8
Europe           5
Middle-east      4
Name: count, dtype: int64
